In [1]:
%load_ext autoreload
%autoreload 2

import os, sys

sys.path.insert(0, "src")

from pandas import DataFrame
from analytics.metrics.metric import MetricCalculator
from analytics.metrics.time import TimeMetrics
from spec.enums import MainTableColumns as Cols
from spec.spec_definition import PS2Versions

In [2]:
from database.config import PS2DataConfig

spec = PS2Versions.v1_0.load()

data_config = PS2DataConfig.from_yaml("sample_data_configs/cwo_s19.yaml", spec)


In [3]:
from analytics.ps2_dataset import PS2Dataset

dataset = PS2Dataset(spec, data_config)

In [4]:
from spec.datatypes import TIMESTAMP_FORMATS
TIMESTAMP_FORMATS

['%Y-%m-%dT%H:%M:%S',
 '%Y-%m-%dT%H:%M:%S.%f%z',
 '%Y-%m-%dT%H:%M:%S%z',
 '%Y-%m-%dT%H:%M:%S.%f']

In [6]:
main_table = dataset.get_main_table()
main_table.head()

,Order,SubjectID,ToolInstances,ServerTimestamp,ServerTimezone,CourseID,CourseSectionID,AssignmentID,ProblemID,Attempt,CodeStateID,IsEventOrderingConsistent,EventType,Score,Compile.Result,CompileMessageType,CompileMessageData,EventID,ParentEventID,SourceLocation
0,0,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20 19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Run.Program,1.0,NaN,NaN,NaN,32-84779,NaN,NaN
1,1,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20 19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Compile,NaN,Success,NaN,NaN,32-84780,32-84779,NaN
2,2,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20 21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Run.Program,1.0,NaN,NaN,NaN,32-84781,NaN,NaN
3,3,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20 21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Compile,NaN,Success,NaN,NaN,32-84782,32-84781,NaN
4,4,4d230b683bf9840553ae57f4acc96e81,Java 8; CodeWorkout,2019-03-20 21:13:09,0,CS 1,1,492.0,32,1,23c57b36002c4d482df3d25f97778f74a2ed581a,True,Run.Program,0.0,NaN,NaN,NaN,32-84783,NaN,NaN


In [7]:
main_table.ServerTimestamp.iloc[0]

Timestamp('2019-03-20 19:34:25')

In [8]:
student_subset = main_table[Cols.SubjectID].unique()[:50]
mt_subset = main_table[main_table[Cols.SubjectID].isin(student_subset)]

In [14]:
time_metrics = TimeMetrics(5*60, 30 * 60, True, Cols.ServerTimestamp)

result = mt_subset.groupby([Cols.SubjectID, Cols.ProblemID]).apply(lambda x: time_metrics.calculate(x))

result

C:\Users\twprice\AppData\Local\Temp\ipykernel_24164\22418638.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = mt_subset.groupby([Cols.SubjectID, Cols.ProblemID]).apply(lambda x: time_metrics.calculate(x))


ActiveTime  IdleTime  TotalTime  \
SubjectID                        ProblemID                                    
04c32d4d95425f73b3a1d6502aed4d48 1                 0.0       0.0        0.0   
                                 3                 0.0       0.0        0.0   
                                 5                 0.0       0.0        0.0   
                                 12                0.0       0.0        0.0   
                                 13                0.0       0.0        0.0   
...                                                ...       ...        ...   
fc5f86251458722c799d1830fa0c2c1f 232               0.0       0.0        0.0   
                                 233               0.0       0.0        0.0   
                                 234               0.0       0.0        0.0   
                                 235               0.0       0.0        0.0   
                                 236               0.0       0.0        0.0   

                                            ActiveTimeAfterCorrect  #Breaks  \
SubjectID                        ProblemID                                    
04c32d4d95425f73b3a1d6502aed4d48 1                             0.0        0   
                                 3                            73.0        0   
                                 5                            56.0        0   
                                 12                            0.0        0   
                                 13                           83.0        0   
...                                                            ...      ...   
fc5f86251458722c799d1830fa0c2c1f 232                         259.0        0   
                                 233                         147.0        0   
                                 234                          12.0        0   
                                 235                           0.0        0   
                                 236                         570.0        0   

                                                     StartTime  \
SubjectID                        ProblemID                       
04c32d4d95425f73b3a1d6502aed4d48 1         2019-02-23 22:44:51   
                                 3         2019-02-23 22:49:34   
                                 5         2019-02-23 22:18:45   
                                 12        2019-02-23 22:54:58   
                                 13        2019-02-23 21:19:38   
...                                                        ...   
fc5f86251458722c799d1830fa0c2c1f 232       2019-02-24 15:05:33   
                                 233       2019-02-21 20:34:04   
                                 234       2019-02-21 19:48:14   
                                 235       2019-02-21 19:53:49   
                                 236       2019-02-21 20:01:41   

                                              FirstCorrectTime  
SubjectID                        ProblemID                      
04c32d4d95425f73b3a1d6502aed4d48 1         2019-02-23 22:44:51  
                                 3         2019-02-23 22:49:34  
                                 5         2019-02-23 22:18:45  
                                 12        2019-02-23 22:54:58  
                                 13        2019-02-23 21:19:38  
...                                                        ...  
fc5f86251458722c799d1830fa0c2c1f 232       2019-02-24 15:05:33  
                                 233       2019-02-21 20:34:04  
                                 234       2019-02-21 19:48:14  
                                 235       2019-02-21 19:53:49  
                                 236       2019-02-21 20:01:41  

[2364 rows x 7 columns]

In [13]:
type(result)

pandas.core.series.Series